In [2]:
# import pandas and wbdata
import pandas as pd
import wbdata as wb

In [29]:
# Read the CSVs into dataframes
df_a = pd.read_csv('Resources/inpe_brazilian_amazon_fires_1999_2019.csv')
df_b = pd.read_csv('Resources/def_area_2004_2019.csv')

# Create a mapping dictionary
state_mapping = {
    'AC': 'ACRE',
    'AM': 'AMAZONAS',
    'AP': 'AMAPA',
    'MA': 'MARANHAO',
    'MT': 'MATO GROSSO',
    'PA': 'PARA',
    'RO': 'RONDONIA',
    'RR': 'RORAIMA',
    'TO': 'TOCANTINS'
}

# Map the full state names in df_a to their initials
df_a['state_initials'] = df_a['state'].map({v: k for k, v in state_mapping.items()})

# Convert the 'year' column in both dataframes to the same type
df_a['year'] = df_a['year'].astype(str)
df_b['Ano/Estados'] = df_b['Ano/Estados'].astype(str)

# Melt df_b to long format to facilitate the merge
df_b_melted = pd.melt(df_b, id_vars=['Ano/Estados'], value_vars=state_mapping.keys(), 
                      var_name='state_initials', value_name='deforestation_area')

# Merge the two dataframes on the year and state initials
merged_df = pd.merge(df_a, df_b_melted, left_on=['year', 'state_initials'], right_on=['Ano/Estados', 'state_initials'], how='outer')

# Filter to keep only data from 2004 onwards
merged_df = merged_df[merged_df['year'] >= '2004']

# Drop the state initials column and the duplicate 'Ano/Estados' column
merged_df.drop(['state_initials', 'Ano/Estados'], axis=1, inplace=True)

merged_df.head()


,year,month,state,latitude,longitude,firespots,deforestation_area
455,2004,1,AMAPA,0.131862,-51.642086,58,46.0
456,2004,2,AMAPA,0.686000,-50.987000,1,46.0
457,2004,4,AMAPA,-0.930000,-52.327000,1,46.0
458,2004,5,AMAPA,-0.919000,-52.286000,1,46.0
459,2004,6,AMAPA,1.473857,-51.059571,7,46.0
